In [14]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from scipy.spatial import distance

result= []
indel_cost = 1
#수입없음 : 0, 0~1천 : 1, 1~2천 : 2, 2~3천 :3, 3~4천 : 4, 4~5천 : 5, 5~6천 : 6, 6~7천 : 7, 7~8천 : 8, 8~9천 : 9, 9~1억 : 10, 1억이상 : 11 
sub_array=[]
#교체비용 대칭행렬 만드는 과정
for j in range(11):
    sub_array.extend(np.linspace(1.5,2,11)[:11-j])
sub_matrix = pd.DataFrame(distance.squareform(sub_array))
    
substitution_cost = pd.DataFrame(columns = [0,1,2,3,4,5,6,7,8,9,10,11], index=[0,1,2,3,4,5,6,7,8,9,10,11],
                               data=sub_matrix)
df = pd.read_csv('income_labeling.csv')
df = df.set_index('pid')
sub_matrix.to_csv('cost_matrix.csv', encoding='utf-8')

In [17]:
#최적 비용 구하는 함수
def opt_match(df):
    for i in range(16):
        df.iloc[0,i] = indel_cost*i
        df.iloc[i,0] = indel_cost*i
    for i in range(1,16):
        for j in range(1,16):
            df.iloc[i,j] = np.min([df.iloc[i,j-1]+1,df.iloc[i-1,j]+1,df.iloc[i-1,j-1]+substitution_cost[df.columns[j]][df.index[i]]])   
        opt_cost = df.iloc[i,j]
    return opt_cost

#distance matrix
result = []
for i in range(100):
    for j in range(1+i,100):
        spl_1 = list(df.iloc[i])
        spl_1.insert(0,'-')
        spl_2 = list(df.iloc[j])
        spl_2.insert(0,'-')
        df_2 = pd.DataFrame(columns=spl_1, index= spl_2, data= np.ones([len(spl_1), len(spl_2)]))
        dict = {'paid' : [i,i+j], 'opt_cost' : opt_match(df_2)}
        result.append(dict)



In [18]:
distance_matrix = distance.squareform([result[x]['opt_cost'] for x in range(len(result))])
pd.DataFrame(distance_matrix)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.00,12.00,24.65,15.50,23.55,8.50,9.00,12.50,7.50,9.50,...,22.95,18.10,24.05,7.50,26.45,26.85,4.00,24.40,23.95,22.60
1,12.00,0.00,25.05,9.00,23.10,11.00,3.00,21.15,18.00,10.50,...,23.35,21.55,24.45,18.20,26.85,27.25,13.55,24.80,24.35,23.75
2,24.65,25.05,0.00,24.90,14.80,24.75,24.95,24.10,24.60,24.70,...,18.55,22.00,11.05,23.40,16.75,21.25,24.55,13.30,10.80,17.00
3,15.50,9.00,24.90,0.00,21.10,11.55,10.00,15.40,12.50,9.50,...,21.75,17.95,23.45,16.70,26.70,27.10,15.05,24.65,23.35,20.70
4,23.55,23.10,14.80,21.10,0.00,22.45,23.00,18.00,21.95,23.55,...,15.20,13.55,10.20,20.70,23.45,22.75,22.60,17.00,11.10,9.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,26.85,27.25,21.25,27.10,22.75,26.95,27.15,26.30,26.80,26.90,...,25.65,26.00,22.00,26.45,15.45,0.00,26.75,21.05,22.45,25.25
96,4.00,13.55,24.55,15.05,22.60,8.50,11.00,10.10,5.00,9.50,...,21.40,14.65,23.95,5.50,26.35,26.75,0.00,24.30,23.85,22.50
97,24.40,24.80,13.30,24.65,17.00,24.50,24.70,21.20,24.35,24.45,...,19.70,19.20,8.55,22.55,19.95,21.05,24.30,0.00,12.10,18.55
98,23.95,24.35,10.80,23.35,11.10,24.05,24.25,20.50,23.90,24.00,...,16.95,16.95,10.60,22.40,21.60,22.45,23.85,12.10,0.00,12.50


In [19]:
pd.DataFrame(distance_matrix).to_csv('distance_matrix(100).csv')